# Programmation orientée objet

Comment créer se propres objets en Python. 

## Vocabulaire

En python tout ce que l'on manipule est un objet : 
* Nombre, liste, dictionnaire, ...
* Tableau numpy
* Fonction, module, ....

Un objet possède des **attributs**. Exemple : 

    z = 1.1J
    z.real
    a = np.array([1, 2, 4])
    a.shape
    
Un objet possède des **méthodes**. Exemple :

    a.mean()
    a.conjugate()
    z.conjugate()
    
Les méthodes sont des fonctions attachées à l'objet. La méthode conjugate d'un tableau n'est pas la même que la méthode conjugate d'un complexe

## Comment créer un objet

### Créer un nombre complexe


In [126]:
import cached_property

import numbers
from math import sqrt, atan2, cos, sin, pi

class Complexe(object):
    def __init__(self, real_part, imaginary_part=0):
        self.re = real_part
        self.im = imaginary_part
        
    def conjugate(self):
        z_conj = Complexe(self.re, -self.im)
        return z_conj
    
    def __str__(self):
        if self.im==0:
            return f"{self.re}"
        elif self.im>=0:
            return f"{self.re} + {self.im}J"
        else:
            return f"{self.re} - {-self.im}J"
        
    def __repr__(self):
        return f'Complexe({self.re}, {self.im})'
    
    def __add__(self, other):
        if isinstance(other, numbers.Real):
            return Complexe(self.re + other, self.im)
        if isinstance(other, Complexe):
            return Complexe(self.re + other.re, self.im + other.im)
        return NotImplemented
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __getitem__(self, key):
        if key==0:
            return self.re
        if key==1:
            return self.im
        raise Exception('La clé doit être 1 ou 2')

    @cached_property.cached_property
    def r(self):
        return sqrt(self.re**2 + self.im**2)
    
    @property
    def phi(self):
        return atan2(self.im, self.re)
    
    def __mul__(self, other):
        if isinstance(other, numbres.Real):
            other = Complexe(other, 0)
        return complexe_polaire(self.r*other.r, self.phi+other.phi)

def complexe_polaire(r, phi):
    return Complexe(r*cos(phi), r*sin(phi))


class ImaginairePur(Complexe):
    def __init__(self, im):
        self.im = im
        self.re = 0

    def __repr__(self):
        return f'ImaginairePur({self.im})'
    
z = Complexe(1, 1)
z2 = z.conjugate()
print(z2)
z3 = ImaginairePur(3.2)

1 - 1J


In [123]:
z3.conjugate()

Complexe(0, -3.2)

In [125]:
isinstance(z3, Complexe)

True

In [124]:
z3 + 5

Complexe(5, 3.2)

In [115]:
z.r
z.r

1.4142135623730951

In [116]:
z.__dict__

{'re': 1, 'im': 1, 'r': 1.4142135623730951}

In [103]:
complexe_polaire(1, pi/2)

Complexe(6.123233995736766e-17, 1.0)

In [68]:
print(z + z2)
print(z + 15)
print(15 + z)
z

2
16 + 2J
16 + 2J


Complexe(1, 2)

In [71]:
z[0]
z[1]

2

In [73]:
z[1] = 5

TypeError: 'Complexe' object does not support item assignment

In [60]:
# a + b
# resultat = a.__add__(b)
# si resultat est NotImplemented alors b.__radd__(a) sinon renvoie resultat

TypeError: unsupported operand type(s) for +: 'Complexe' and 'str'

In [57]:
1 + 'poipoi'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [61]:
a = np.array([1, 2, 5])
a

array([1, 2, 5])

In [63]:
print(a)

[1 2 5]


## Nouveaux concepts



### Méthodes spéciales

* `__init__`
* `__repr__`, `__str__`

Lorsque c'est possible, `__repr__` doit représenté au mieux l'objet. Souvent il s'agit d'une chaine de caractère qui en étant évaluée renvoie un objet similaire. La méthode `__str__` renvoie `__repr__` par défaut. Sinon, elle doit être plus simple.

Opérateur unaire et binaire
* `__neg__`
* `__add__`, `__sub__`, `__mul__`, `__truediv__`, `__mod__`, `__pow__`
* `__radd__`, ...
* `__eq__` (==), `__ne__` (!=), `__lt__` (<), `__le__` (<=), `__gt__`, `__ge__`
* `__or__`, `__and__`, `__xor__`


Emulateur de contenu
* a[key] => `a.__getitem__(key)`
* a[key] = val => `a.__setitem__(key, val)`
* del a[key] => `a.__delitem__(key)`
* len(a) => `a.__len__()`
* for elm in a => `for elm in a.__iter__()`

### Attributs et property
* Attributs de class vs attributs d'objets
* property

### Héritage
* isinstance permet de tester si un objet est une instance d'une classe. 

In [87]:
class Test:
    a = 1
#    b = []
    def __init__(self):
        self.b = []
    def ma_methode(self):
        return 1
    
    def ajoute(self, val):
        self.b.append(val)
    
Test.ma_methode
Test.a

t = Test()
t.a
print(t.__dict__)
t.a = 4
print(t.__dict__)
print(Test.a)

{'b': []}
{'b': [], 'a': 4}
1


In [89]:
t1 = Test()
t2 = Test()
t1.ajoute(3)
print(t2.b)

[]


### Circuit électrique

![circuit](kicad.sch.png "Exemple de circuit")

Objectif : faire comprendre à Python ce circuit pour pouvoir ensuite faire des calculs. Ici, on demandera de calculer l'impédance complexe à une fréquence donnée.

Stucture en arbre : 
<code>
Serie :
    |- Parallele:
    |   |-inductance
    |   |-resistance
    |   |-condensateur
    |- resistance
</code>


Il y a plusieurs objets de nature différente donc de classe différente (résistance, condensateur, circuit parallèle, ...). Mais ces objets sont tous des circuits bibolaires. Tous ces objets devront mettre en oeuvre un méthode pour calculer leur impédance à une fréquence donnée. 

Code final en Python (objectif à atteindre pour que l'objet soit le plus simple à utiliser):

    R1 = Resistance(10)
    R1 = Resistance(5)
    L1 = Inductance(15E-6)
    C1 = Condensateur(10E-6)

    circuit = R2 + (L1|R1|C1)
    
    print(circuit.impedance(50))
    isinstance(R1, CircuitBibolaire) # True
    isinstance(circuit, CircuitBibolaire) # True    




### Simulation autour d'un pendule